<a href="https://colab.research.google.com/github/brenolucas00/exerc-ciosawariwebscrapping/blob/main/Realiza%C3%A7%C3%A3o_de_Web_Scraping_dos_dados_de_im%C3%B3veis_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [2]:
requests.get('https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145').content

b'\n<!DOCTYPE html>\n<html lang="pt-BR">\n<head><link rel="preconnect" href="https://www.google-analytics.com"/><link rel="preconnect" href="https://www.google.com"/><link rel="preconnect" href="https://data.mercadolibre.com"/><link rel="preconnect" href="https://http2.mlstatic.com"/><link rel="preconnect" href="https://stats.g.doubleclick.net"/><link rel="preconnect" href="https://analytics.mercadolivre.com.br"/><link rel="preconnect" href="https://analytics.mercadolivre.com"/><link rel="preconnect" href="https://www.google.com.br"/><script type=\'text/javascript\'>window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-689ffbd95eae88e39ac","applicationID":"1729522169","applicationTime":1448.208866,"transactionName":"YlZQYEVZC0QEV0BZV1scd0xHSgBEFl5HH39wZx0bHQ==","queueTime":0,"ttGuid":"d6be38a21e57663b","agentToken":null}; (window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-dat

In [3]:
ZONAS = ['norte', 'sul', 'leste', 'oeste']
PAGINAS = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']
URL_ML = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'

# Podemos definir dois padrões regex, um para o numero de quartos e outro para a area
re_quarto = '\| (.*) quarto'
re_area = '> (.*) m²'
re_quarto_area_ausente = "> (.*) quarto"

In [4]:
URL_ML.format('norte', '')

'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/'

In [17]:
def coletar_dados_ml(url, zona):
  c = requests.get(url).content
  soup = BeautifulSoup(c)
  atributos = soup.find_all('div', {"class": "item__attrs"})
  precos = soup.find_all('span', {"class": "price__fraction"})

  lista_precos = []
  for s in precos:
    p = re.findall('<span class="price__fraction">(.*)</span>', str(s))
    lista_precos.append(int(p[0].replace('.','')))


  lista_areas = []
  lista_quartos = []
  lista_zonas = []
  for a in atributos:
   
    n_quartos = re.findall(re_quarto, str(a))
    area = re.findall(re_area, str(a))
   
    if n_quartos==[]:
      n_quartos = re.findall(re_quarto_area_ausente, str(a))

      if n_quartos==[]:
        n_quartos=np.nan
      else:
        n_quartos=n_quartos[0]
  
    else:
        n_quartos=n_quartos[0]

    if area==[]:
      area=np.nan
    else:
      area=area[0]
   

   
    lista_quartos.append(n_quartos)
    lista_areas.append(area)
    lista_zonas.append(zona)

  dados = {
      'zona': lista_zonas,
      'quartos': lista_quartos,
      'area': lista_areas,
      'preco': lista_precos
      }

  df = pd.DataFrame(dados)
  return df

In [18]:
lista_dfs = []
for zona in ZONAS:
  for pg in PAGINAS:
    url = URL_ML.format(zona, pg)
    print('Carregando a URL', url)

    df = coletar_dados_ml(url, zona)
    lista_dfs.append(df)
    sleep(1)
    

Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Carregando a URL https://imoveis.mercadolivre.com

In [19]:
dados = pd.concat(lista_dfs)

In [20]:
dados

,zona,quartos,area,preco


In [21]:
dados['quartos'] = dados['quartos'].astype(float)
dados['area'] = dados['area'].astype(float)

In [22]:
dados.to_csv('dados_calculadora_imoveis_aula.csv', index=False)

In [23]:
pd.read_csv("dados_calculadora_imoveis_aula.csv")

,zona,quartos,area,preco


In [24]:
ls

dados_calculadora_imoveis_aula.csv  sample_data/
